# SEBI Fraud Detection System with AI Multi-Agent Workflow

## Investment Fraud Prevention & Detection Platform

This system leverages **LlamaIndex** multi-agent workflows with **Gemini 2.5 Pro** to create a comprehensive fraud detection platform that:

1. **Scans online platforms** for suspicious investment offers, deepfake content, and fraudulent advisors
2. **Monitors social media** for misleading stock tips and pump-and-dump schemes  
3. **Analyzes corporate announcements** for authenticity and credibility scoring
4. **Verifies advisor credentials** against regulatory databases
5. **Provides real-time alerts** and risk assessments for retail investors

The system uses advanced AI agents to protect retail investors from securities market fraud, aligned with SEBI's Safe Space initiative.

**Requirements**: You need a Gemini API key from Google AI Studio. The system uses LlamaIndex for multi-agent orchestration.

In [ ]:
# Install required packages for fraud detection system
%pip install llama-index llama-index-utils-workflow llama-index-llms-google-genai llama-index-tools-google
%pip install beautifulsoup4 requests pandas numpy scikit-learn nltk textblob
%pip install streamlit plotly dash flask sqlalchemy
%pip install opencv-python pillow librosa tensorflow
%pip install youtube-dl tweepy selenium webdriver-manager
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Setup Gemini 2.5 Pro for Fraud Detection

The fraud detection system uses **Gemini 2.5 Pro** as the core reasoning engine for all AI agents. Each agent specializes in different types of fraud detection:

- **Content Analysis Agent**: Analyzes text, images, and videos for fraud indicators
- **Advisor Verification Agent**: Cross-references advisor credentials with regulatory databases  
- **Social Media Monitor Agent**: Scans platforms for suspicious investment schemes
- **Corporate Announcement Agent**: Validates authenticity of company announcements
- **Risk Assessment Agent**: Provides overall fraud risk scoring

Make sure to set your Gemini API key as an environment variable (`GEMINI_API_KEY` or `GOOGLE_API_KEY`).

In [2]:
import os
import json
import re
import requests
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
from urllib.parse import urlparse
import sqlite3

# Web scraping and content analysis
from bs4 import BeautifulSoup
from textblob import TextBlob
import nltk

# Machine learning for fraud detection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Optionally load .env if python-dotenv is installed
try:
    from dotenv import load_dotenv
    # load .env from the notebook directory (if present)
    notebook_dir = Path().resolve()
    load_dotenv(dotenv_path=notebook_dir / '.env')
except Exception:
    pass

from llama_index.llms.google_genai import GoogleGenAI

# Read API key from environment; support both GEMINI_API_KEY (user .env) and GOOGLE_API_KEY
api_key = os.environ.get('GEMINI_API_KEY') or os.environ.get('GOOGLE_API_KEY')
if not api_key:
    raise RuntimeError("Missing Gemini API key. Set the GEMINI_API_KEY or GOOGLE_API_KEY environment variable to your API key.")

# Initialize LLM for fraud detection system
llm = GoogleGenAI(model="gemini-2.5-flash", api_key=api_key)

print("✅ Fraud Detection System Initialized Successfully!")

✅ Fraud Detection System Initialized Successfully!


## Build Fraud Detection Tools & Database

The fraud detection system uses multiple specialized tools and maintains databases for:

1. **Regulatory Database**: SEBI registered advisors, brokers, and intermediaries
2. **Fraud Patterns Database**: Known fraud indicators and suspicious content patterns  
3. **Market Data Integration**: Real-time stock prices and trading volumes
4. **Social Media Monitoring**: Scanning WhatsApp, Telegram, Twitter for suspicious activities
5. **Content Analysis**: Deepfake detection, document verification, and authenticity checks

We'll use Gemini's built-in Google Search capability along with custom tools for comprehensive fraud detection.

In [3]:
from google.genai import types

# Create Google Search tool for fraud detection
google_search_tool = types.Tool(
    google_search=types.GoogleSearch()
)

# Enhanced LLM with search capabilities for fraud detection
llm_with_search = GoogleGenAI(
    model="gemini-2.5-flash",
    api_key=api_key,
    generation_config=types.GenerateContentConfig(tools=[google_search_tool])
)

# Initialize fraud detection databases
def init_fraud_databases():
    """Initialize SQLite databases for fraud detection system"""
    
    # Create regulatory database
    conn = sqlite3.connect('../data/regulatory_db/sebi_advisors.db')
    cursor = conn.cursor()
    
    # Table for registered advisors
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS registered_advisors (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            registration_number TEXT UNIQUE,
            category TEXT,
            validity_date TEXT,
            status TEXT,
            contact_info TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Table for fraud reports
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS fraud_reports (
            id INTEGER PRIMARY KEY,
            content_url TEXT,
            fraud_type TEXT,
            risk_score REAL,
            detected_patterns TEXT,
            verification_status TEXT,
            reported_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Table for suspicious social media content
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS social_media_alerts (
            id INTEGER PRIMARY KEY,
            platform TEXT,
            content TEXT,
            user_handle TEXT,
            fraud_indicators TEXT,
            risk_level TEXT,
            detected_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    conn.commit()
    conn.close()
    print("✅ Fraud detection databases initialized!")

# Initialize the databases
init_fraud_databases()

✅ Fraud detection databases initialized!


## Test Fraud Detection Capabilities

Let's test the system's ability to identify potential investment fraud by analyzing suspicious content and verifying information:

In [4]:
# Test fraud detection capabilities
test_query = """
Analyze this suspicious investment offer: 'Get 500% returns in 30 days! 
Guaranteed profits with our secret trading algorithm. 
Contact advisor Raj Kumar (License: SEBI/IA/2023/FAKE123) 
Join our exclusive WhatsApp group for insider tips!'

Please identify fraud indicators and verify the advisor credentials.
"""

response = llm_with_search.complete(test_query)
print("🚨 FRAUD DETECTION ANALYSIS:")
print("=" * 50)
print(response)
print("=" * 50)

🚨 FRAUD DETECTION ANALYSIS:
This investment offer contains multiple significant fraud indicators, and the provided advisor credentials appear to be illegitimate.

**Fraud Indicators:**

1.  **Unrealistic Returns:** The promise of "500% returns in 30 days" is an extreme and highly improbable claim for any legitimate investment. Such high, short-term returns are a classic characteristic of Ponzi schemes or other investment scams, as genuine investments always carry a degree of risk and rarely yield such rapid and substantial profits.
2.  **Guaranteed Profits:** All legitimate investments carry inherent risks, and no financial advisor can truthfully "guarantee profits." This assurance is a major red flag, as it misleads potential investors by eliminating the reality of market fluctuations and potential losses.
3.  **"Secret Trading Algorithm":** The use of vague terms like a "secret trading algorithm" without providing any verifiable details or a track record is a common tactic employed b

## Specialized Fraud Detection Agents

The fraud detection system employs specialized AI agents that work together:

### 🔍 **Content Scanner Agent**
- Scans websites, social media, and documents for fraud indicators
- Analyzes text patterns, promises of unrealistic returns, urgency tactics
- Detects deepfake videos/audios and manipulated documents

### ✅ **Advisor Verification Agent** 
- Cross-references advisor credentials with SEBI regulatory database
- Validates registration numbers, licenses, and validity periods
- Flags impersonators and unregistered entities

### 📱 **Social Media Monitor Agent**
- Monitors WhatsApp groups, Telegram channels, Twitter for suspicious tips
- Detects pump-and-dump schemes and coordinated market manipulation
- Tracks viral fraud content and fake trading apps

### 📊 **Corporate Announcement Analyzer**
- Verifies authenticity of company announcements
- Cross-checks with historical filings and counterparty disclosures  
- Provides credibility scoring based on company performance vs claims

### ⚠️ **Risk Assessment Agent**
- Aggregates findings from all agents
- Generates comprehensive fraud risk scores
- Creates actionable alerts for investors and regulators

Each agent maintains state and can collaborate to provide comprehensive fraud detection.

In [5]:
from llama_index.core.workflow import Context
import sqlite3
import json
from datetime import datetime

# Fraud Detection Tools
async def scan_content_for_fraud(ctx: Context, content: str, content_type: str = "text") -> str:
    """Scans content (text/URL/document) for fraud indicators and suspicious patterns"""
    
    fraud_indicators = [
        "guaranteed returns", "secret algorithm", "insider tips", "risk-free", 
        "limited time offer", "exclusive opportunity", "500% returns", "get rich quick",
        "investment guru", "proven strategy", "urgent action required", "act now"
    ]
    
    detected_patterns = []
    risk_score = 0
    
    content_lower = content.lower()
    for indicator in fraud_indicators:
        if indicator in content_lower:
            detected_patterns.append(indicator)
            risk_score += 10
    
    # Use LLM for advanced analysis
    analysis_prompt = f"""
    Analyze this {content_type} content for investment fraud indicators:
    
    Content: {content}
    
    Look for:
    1. Unrealistic return promises
    2. Pressure tactics and urgency
    3. Lack of proper disclaimers
    4. Unverified credentials
    5. Social media manipulation tactics
    
    Provide a detailed fraud risk assessment.
    """
    
    response = await llm_with_search.acomplete(analysis_prompt)
    
    # Store in database
    current_state = await ctx.store.get("state")
    if "fraud_scans" not in current_state:
        current_state["fraud_scans"] = []
    
    scan_result = {
        "content": content[:200] + "..." if len(content) > 200 else content,
        "type": content_type,
        "detected_patterns": detected_patterns,
        "risk_score": min(risk_score, 100),
        "ai_analysis": str(response),
        "timestamp": datetime.now().isoformat()
    }
    
    current_state["fraud_scans"].append(scan_result)
    await ctx.store.set("state", current_state)
    
    return f"Content scanned. Risk Score: {scan_result['risk_score']}/100. Patterns detected: {detected_patterns}"

async def verify_advisor_credentials(ctx: Context, advisor_name: str, registration_number: str = None) -> str:
    """Verifies advisor credentials against SEBI regulatory database"""
    
    # Search SEBI database (simulated - in production, integrate with actual SEBI APIs)
    verification_prompt = f"""
    Verify the credentials of investment advisor: {advisor_name}
    Registration Number: {registration_number if registration_number else "Not provided"}
    
    Search for:
    1. SEBI registration status
    2. Valid license and category
    3. Any disciplinary actions
    4. Contact information verification
    5. Recent regulatory updates
    
    Provide verification status and any red flags.
    """
    
    response = await llm_with_search.acomplete(verification_prompt)
    
    # Store verification result
    current_state = await ctx.store.get("state")
    if "advisor_verifications" not in current_state:
        current_state["advisor_verifications"] = []
    
    verification_result = {
        "advisor_name": advisor_name,
        "registration_number": registration_number,
        "verification_result": str(response),
        "timestamp": datetime.now().isoformat()
    }
    
    current_state["advisor_verifications"].append(verification_result)
    await ctx.store.set("state", current_state)
    
    return f"Advisor verification completed for {advisor_name}. Check results in verification database."

async def monitor_social_media(ctx: Context, platform: str, search_terms: str) -> str:
    """Monitors social media platforms for suspicious investment content"""
    
    monitoring_prompt = f"""
    Monitor {platform} for suspicious investment content related to: {search_terms}
    
    Look for:
    1. Pump and dump schemes
    2. Fake trading apps promotion
    3. Impersonation of legitimate advisors
    4. Coordinated market manipulation
    5. Misleading stock tips
    6. IPO allotment frauds
    
    Identify patterns, user behaviors, and potential fraud networks.
    """
    
    response = await llm_with_search.acomplete(monitoring_prompt)
    
    # Store monitoring results
    current_state = await ctx.store.get("state")
    if "social_media_monitoring" not in current_state:
        current_state["social_media_monitoring"] = []
    
    monitoring_result = {
        "platform": platform,
        "search_terms": search_terms,
        "findings": str(response),
        "timestamp": datetime.now().isoformat()
    }
    
    current_state["social_media_monitoring"].append(monitoring_result)
    await ctx.store.set("state", current_state)
    
    return f"Social media monitoring completed for {platform}. Suspicious activities logged."

async def analyze_corporate_announcement(ctx: Context, announcement_text: str, company_name: str) -> str:
    """Analyzes corporate announcements for authenticity and credibility"""
    
    analysis_prompt = f"""
    Analyze this corporate announcement for authenticity and credibility:
    
    Company: {company_name}
    Announcement: {announcement_text}
    
    Verify:
    1. Official company communication channels
    2. Consistency with historical performance
    3. Regulatory filing requirements
    4. Cross-verification with exchange announcements
    5. Credibility of claimed partnerships/deals
    6. Market impact plausibility
    
    Provide credibility score (1-10) and authenticity assessment.
    """
    
    response = await llm_with_search.acomplete(analysis_prompt)
    
    # Store analysis
    current_state = await ctx.store.get("state")
    if "announcement_analysis" not in current_state:
        current_state["announcement_analysis"] = []
    
    analysis_result = {
        "company_name": company_name,
        "announcement": announcement_text[:300] + "..." if len(announcement_text) > 300 else announcement_text,
        "analysis": str(response),
        "timestamp": datetime.now().isoformat()
    }
    
    current_state["announcement_analysis"].append(analysis_result)
    await ctx.store.set("state", current_state)
    
    return f"Corporate announcement analyzed for {company_name}. Credibility assessment completed."

async def generate_fraud_report(ctx: Context, report_type: str = "comprehensive") -> str:
    """Generates comprehensive fraud detection report with risk assessments"""
    
    current_state = await ctx.store.get("state")
    
    # Compile all findings
    report_data = {
        "fraud_scans": current_state.get("fraud_scans", []),
        "advisor_verifications": current_state.get("advisor_verifications", []),
        "social_media_monitoring": current_state.get("social_media_monitoring", []),
        "announcement_analysis": current_state.get("announcement_analysis", [])
    }
    
    report_prompt = f"""
    Generate a comprehensive fraud detection report based on these findings:
    
    {json.dumps(report_data, indent=2)}
    
    Include:
    1. Executive Summary of fraud risks identified
    2. High-risk content and advisors flagged
    3. Social media threats and manipulation attempts
    4. Corporate announcement authenticity issues
    5. Actionable recommendations for investors
    6. Regulatory alerts and compliance issues
    7. Risk mitigation strategies
    
    Format as a professional security report for SEBI and retail investors.
    """
    
    response = await llm_with_search.acomplete(report_prompt)
    
    current_state["fraud_report"] = {
        "report_content": str(response),
        "generated_at": datetime.now().isoformat(),
        "report_type": report_type
    }
    await ctx.store.set("state", current_state)
    
    return f"Comprehensive fraud detection report generated. Report contains analysis of all detected threats."

print("✅ Fraud Detection Tools Initialized Successfully!")

✅ Fraud Detection Tools Initialized Successfully!


## Build Multi-Agent Fraud Detection System

The fraud detection system consists of specialized agents working collaboratively:

### 🔍 **Content Scanner Agent**
- **Primary Role**: Identifies and analyzes suspicious investment content
- **Capabilities**: Text analysis, pattern recognition, fraud indicator detection
- **Handoff**: Can escalate to Advisor Verification Agent for credential checks

### ✅ **Advisor Verification Agent** 
- **Primary Role**: Validates investment advisor credentials and legitimacy
- **Capabilities**: SEBI database queries, license verification, background checks
- **Handoff**: Works with Social Media Monitor for impersonation detection

### 📱 **Social Media Monitor Agent**
- **Primary Role**: Scans social platforms for fraudulent investment schemes
- **Capabilities**: Platform monitoring, viral content analysis, network mapping
- **Handoff**: Coordinates with Corporate Announcement Analyzer for cross-verification

### 📊 **Corporate Announcement Analyzer**
- **Primary Role**: Verifies authenticity of company announcements and press releases
- **Capabilities**: Document verification, historical analysis, credibility scoring
- **Handoff**: Provides data to Risk Assessment Agent for final evaluation

### ⚠️ **Risk Assessment Agent**
- **Primary Role**: Generates comprehensive fraud reports and risk scores
- **Capabilities**: Data aggregation, threat analysis, recommendation generation
- **Output**: Final fraud detection reports for regulators and investors

Each agent maintains specialized knowledge and can collaborate seamlessly to provide comprehensive protection against investment fraud.

In [6]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent

# 🔍 Content Scanner Agent
content_scanner_agent = FunctionAgent(
    name="ContentScannerAgent",
    description="Scans and analyzes content for investment fraud indicators, suspicious patterns, and deceptive tactics.",
    system_prompt=(
        "You are the ContentScannerAgent specialized in detecting investment fraud. "
        "Analyze all content for fraud indicators like unrealistic returns, pressure tactics, fake credentials, and deceptive language. "
        "When suspicious advisors are mentioned, hand off to AdvisorVerificationAgent for credential verification. "
        "Focus on protecting retail investors from fraudulent schemes and misleading content."
    ),
    llm=llm,
    tools=[scan_content_for_fraud],
    can_handoff_to=["AdvisorVerificationAgent", "SocialMediaMonitorAgent"],
)

# ✅ Advisor Verification Agent
advisor_verification_agent = FunctionAgent(
    name="AdvisorVerificationAgent", 
    description="Verifies investment advisor credentials against SEBI regulatory database and flags impersonators.",
    system_prompt=(
        "You are the AdvisorVerificationAgent responsible for validating investment advisor credentials. "
        "Cross-reference all advisor claims with SEBI registration database. "
        "Flag unregistered entities, expired licenses, and impersonators. "
        "Work with SocialMediaMonitorAgent to track advisor impersonation across platforms. "
        "Ensure only legitimate advisors are trusted by retail investors."
    ),
    llm=llm,
    tools=[verify_advisor_credentials],
    can_handoff_to=["SocialMediaMonitorAgent", "RiskAssessmentAgent"],
)

# 📱 Social Media Monitor Agent  
social_media_monitor_agent = FunctionAgent(
    name="SocialMediaMonitorAgent",
    description="Monitors social media platforms for fraudulent investment schemes, pump-and-dump activities, and manipulative content.",
    system_prompt=(
        "You are the SocialMediaMonitorAgent that monitors social platforms for investment fraud. "
        "Detect pump-and-dump schemes, fake trading apps, WhatsApp/Telegram fraud groups, and viral misinformation. "
        "Track coordinated manipulation attempts and identify fraud networks. "
        "Hand off corporate announcement verification to CorporateAnnouncementAnalyzer when company-related fraud is detected. "
        "Protect investors from social media-based financial fraud."
    ),
    llm=llm,
    tools=[monitor_social_media],
    can_handoff_to=["CorporateAnnouncementAnalyzer", "RiskAssessmentAgent"],
)

# 📊 Corporate Announcement Analyzer
corporate_announcement_analyzer = FunctionAgent(
    name="CorporateAnnouncementAnalyzer",
    description="Analyzes corporate announcements for authenticity, verifies claims, and provides credibility scoring.",
    system_prompt=(
        "You are the CorporateAnnouncementAnalyzer that verifies corporate announcements and press releases. "
        "Cross-check announcements with historical filings, counterparty disclosures, and regulatory databases. "
        "Identify fabricated news, misleading claims, and market manipulation attempts. "
        "Provide credibility scores and flag suspicious announcements that could mislead investors. "
        "Hand off to RiskAssessmentAgent for final fraud risk evaluation."
    ),
    llm=llm,
    tools=[analyze_corporate_announcement],
    can_handoff_to=["RiskAssessmentAgent"],
)

# ⚠️ Risk Assessment Agent
risk_assessment_agent = FunctionAgent(
    name="RiskAssessmentAgent",
    description="Generates comprehensive fraud reports, risk scores, and actionable recommendations for investors and regulators.",
    system_prompt=(
        "You are the RiskAssessmentAgent that provides final fraud risk assessment and generates comprehensive reports. "
        "Aggregate findings from all agents to create detailed fraud analysis reports. "
        "Provide risk scores, investor warnings, and regulatory recommendations. "
        "Generate actionable insights for SEBI, exchanges, and retail investors to prevent financial fraud. "
        "Your reports should be clear, comprehensive, and focused on investor protection."
    ),
    llm=llm,
    tools=[generate_fraud_report],
    can_handoff_to=["ContentScannerAgent"],  # Can restart investigation cycle
)

print("✅ All Fraud Detection Agents Created Successfully!")
print("\n🤖 Available Agents:")
print("- ContentScannerAgent: Content fraud detection")
print("- AdvisorVerificationAgent: Credential verification")  
print("- SocialMediaMonitorAgent: Platform monitoring")
print("- CorporateAnnouncementAnalyzer: Announcement verification")
print("- RiskAssessmentAgent: Risk scoring & reporting")

✅ All Fraud Detection Agents Created Successfully!

🤖 Available Agents:
- ContentScannerAgent: Content fraud detection
- AdvisorVerificationAgent: Credential verification
- SocialMediaMonitorAgent: Platform monitoring
- CorporateAnnouncementAnalyzer: Announcement verification
- RiskAssessmentAgent: Risk scoring & reporting


## Initialize Fraud Detection Workflow

Now we'll create the **AgentWorkflow** that orchestrates all fraud detection agents. The workflow starts with the **ContentScannerAgent** as the root agent, which analyzes incoming content and hands off to specialized agents as needed.

The workflow maintains state across all fraud detection activities and ensures comprehensive analysis of potential threats.

In [8]:
from llama_index.core.agent.workflow import AgentWorkflow

# Create Fraud Detection Workflow
fraud_detection_workflow = AgentWorkflow(
    agents=[
        content_scanner_agent,
        advisor_verification_agent, 
        social_media_monitor_agent,
        corporate_announcement_analyzer,
        risk_assessment_agent
    ],
    root_agent=content_scanner_agent.name,
    initial_state={
        "fraud_scans": [],
        "advisor_verifications": [],
        "social_media_monitoring": [],
        "announcement_analysis": [],
        "fraud_report": {
            "status": "Initialized",
            "total_threats_detected": 0,
            "high_risk_items": [],
            "recommendations": []
        },
        "session_metadata": {
            "started_at": datetime.now().isoformat(),
            "session_id": f"fraud_detection_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            "platform": "SEBI Fraud Detection System",
            "version": "1.0.0"
        }
    },
)

print("🚨 SEBI Fraud Detection System Ready!")
print("=" * 50)
print("📊 System Status: Operational")
print(f"🔍 Active Agents: {len(fraud_detection_workflow.agents)}")
print(f"🛡️ Root Agent: {fraud_detection_workflow.root_agent}")
print("📱 Monitoring: Social Media, Websites, Announcements")
print("✅ Database: Regulatory verification enabled") 
print("🤖 AI Engine: Gemini 2.5 Pro")
print("=" * 50)

🚨 SEBI Fraud Detection System Ready!
📊 System Status: Operational
🔍 Active Agents: 5
🛡️ Root Agent: ContentScannerAgent
📱 Monitoring: Social Media, Websites, Announcements
✅ Database: Regulatory verification enabled
🤖 AI Engine: Gemini 2.5 Pro


## Execute Fraud Detection Analysis

The system will now analyze a real-world fraud scenario involving multiple threat vectors:
- Suspicious investment offers with unrealistic returns
- Unverified advisor credentials  
- Social media manipulation tactics
- Potentially fake corporate announcements

Watch as different agents collaborate to provide comprehensive fraud detection and risk assessment.

In [9]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

# Comprehensive Fraud Detection Scenario
fraud_scenario = """
🚨 URGENT FRAUD DETECTION REQUEST 🚨

Analyze the following multi-layered fraud scenario:

1. SUSPICIOUS INVESTMENT OFFER:
"🔥 GUARANTEED 800% RETURNS in 45 days! 🔥
Join our EXCLUSIVE trading group with SECRET ALGORITHMS used by billionaires!
✅ Risk-FREE investment ✅ No experience needed
📞 Contact certified advisor: Rajesh Sharma (SEBI Reg: IA/2024/FAKE999)
💎 Limited slots available - ACT NOW or MISS OUT FOREVER!"

2. SOCIAL MEDIA MANIPULATION:
WhatsApp group "💰 Mumbai Stock Tips 💰" with 5000+ members sharing:
"🚀 URGENT: Buy XYZ Corp before announcement tomorrow! 
Insider information confirms 500% jump! 
Trust me, I made ₹50 lakhs last week! 
Delete after reading!"

3. SUSPICIOUS CORPORATE ANNOUNCEMENT:
"ABC Industries announces revolutionary AI partnership with Google and Microsoft.
Expected revenue impact: ₹10,000 crores in next quarter.
Stock price target revised to ₹5000 (current: ₹100)
Press release signed by CEO John Smith"

4. FAKE TRADING APP:
"📱 Download TradeGuru Pro - India's #1 trading platform!
⭐ 5-star ratings ⭐ Used by 10 lakh+ investors
🎁 Sign up bonus: ₹10,000 free trading credits
✨ Developed by IIT graduates, approved by RBI"

TASK: Conduct comprehensive fraud analysis covering all aspects.
"""

print("🚨 INITIATING COMPREHENSIVE FRAUD DETECTION ANALYSIS")
print("=" * 70)

# Execute the fraud detection workflow
handler = fraud_detection_workflow.run(user_msg=fraud_scenario)

current_agent = None
threat_count = 0

async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'🔥' if 'Scanner' in current_agent else '✅' if 'Verification' in current_agent else '📱' if 'Monitor' in current_agent else '📊' if 'Analyzer' in current_agent else '⚠️'} {'='*15} {current_agent} {'='*15}")
        
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print(f"📤 Analysis: {event.response.content[:200]}...")
        if event.tool_calls:
            tool_names = [call.tool_name for call in event.tool_calls]
            print(f"🛠️  Using Tools: {', '.join(tool_names)}")
            
    elif isinstance(event, ToolCallResult):
        threat_count += 1
        print(f"🔧 Tool: {event.tool_name}")
        print(f"   Input: {str(event.tool_kwargs)[:100]}...")
        print(f"   Result: {str(event.tool_output)[:150]}...")
        
    elif isinstance(event, ToolCall):
        print(f"🔨 Executing: {event.tool_name}")

print(f"\n🚨 FRAUD DETECTION COMPLETED - {threat_count} THREATS ANALYZED")
print("=" * 70)

🚨 INITIATING COMPREHENSIVE FRAUD DETECTION ANALYSIS

🔥 =============== ContentScannerAgent ===============
🛠️  Using Tools: scan_content_for_fraud, scan_content_for_fraud, scan_content_for_fraud, scan_content_for_fraud
🔨 Executing: scan_content_for_fraud
🔨 Executing: scan_content_for_fraud
🔨 Executing: scan_content_for_fraud
🔨 Executing: scan_content_for_fraud
🔧 Tool: scan_content_for_fraud
   Input: {'content': '🚀 URGENT: Buy XYZ Corp before announcement tomorrow!\nInsider information confirms 500%...
   Result: Content scanned. Risk Score: 0/100. Patterns detected: []...
🔧 Tool: scan_content_for_fraud
   Input: {'content_type': 'text', 'content': '🔥 GUARANTEED 800% RETURNS in 45 days! 🔥\nJoin our EXCLUSIVE tra...
   Result: Content scanned. Risk Score: 30/100. Patterns detected: ['secret algorithm', 'risk-free', 'act now']...
🔧 Tool: scan_content_for_fraud
   Input: {'content': "📱 Download TradeGuru Pro - India's #1 trading platform!\n⭐ 5-star ratings ⭐ Used by 10 ...
   Result: Cont

## Final Fraud Detection Report

Generate the comprehensive fraud detection report with all findings, risk assessments, and actionable recommendations for SEBI, exchanges, and retail investors.

In [10]:
# Generate and display comprehensive fraud detection report
state = await handler.ctx.store.get("state")

print("🚨" + "="*80 + "🚨")
print("             SEBI FRAUD DETECTION SYSTEM - COMPREHENSIVE REPORT")  
print("🚨" + "="*80 + "🚨")

# Display fraud report if generated
if "fraud_report" in state and "report_content" in state["fraud_report"]:
    print("\n📊 EXECUTIVE SUMMARY:")
    print("-" * 50)
    print(state["fraud_report"]["report_content"])

# Display individual analysis results
print(f"\n🔍 CONTENT SCANS PERFORMED: {len(state.get('fraud_scans', []))}")
for i, scan in enumerate(state.get('fraud_scans', []), 1):
    print(f"\n   Scan #{i}:")
    print(f"   ⚠️  Risk Score: {scan['risk_score']}/100")
    print(f"   🔍 Patterns: {', '.join(scan['detected_patterns'])}")
    print(f"   📝 Content: {scan['content']}")

print(f"\n✅ ADVISOR VERIFICATIONS: {len(state.get('advisor_verifications', []))}")
for i, verification in enumerate(state.get('advisor_verifications', []), 1):
    print(f"\n   Verification #{i}:")
    print(f"   👤 Advisor: {verification['advisor_name']}")
    print(f"   📋 Registration: {verification.get('registration_number', 'Not provided')}")

print(f"\n📱 SOCIAL MEDIA MONITORING: {len(state.get('social_media_monitoring', []))}")
for i, monitoring in enumerate(state.get('social_media_monitoring', []), 1):
    print(f"\n   Monitoring #{i}:")
    print(f"   🌐 Platform: {monitoring['platform']}")
    print(f"   🔎 Search Terms: {monitoring['search_terms']}")

print(f"\n📊 CORPORATE ANNOUNCEMENTS ANALYZED: {len(state.get('announcement_analysis', []))}")
for i, analysis in enumerate(state.get('announcement_analysis', []), 1):
    print(f"\n   Analysis #{i}:")
    print(f"   🏢 Company: {analysis['company_name']}")
    print(f"   📄 Announcement: {analysis['announcement']}")

# Session metadata
session_data = state.get('session_metadata', {})
print(f"\n📈 SESSION SUMMARY:")
print("-" * 30)
print(f"🔗 Session ID: {session_data.get('session_id', 'N/A')}")
print(f"⏰ Started: {session_data.get('started_at', 'N/A')}")
print(f"🛡️  Platform: {session_data.get('platform', 'SEBI Fraud Detection System')}")
print(f"🤖 AI Engine: Gemini 2.5 Pro")

# Final recommendations
print(f"\n🎯 KEY RECOMMENDATIONS:")
print("-" * 30)
print("1. 🚫 Block suspicious investment offers with unrealistic return promises")
print("2. ⚠️  Alert investors about unverified advisor credentials")  
print("3. 📱 Monitor and shut down fraudulent social media groups")
print("4. 📊 Verify corporate announcements through official channels")
print("5. 🔍 Implement real-time scanning of all investment platforms")
print("6. 📚 Educate retail investors about common fraud patterns")

print("\n" + "🚨" + "="*80 + "🚨")
print("          END OF FRAUD DETECTION REPORT - STAY VIGILANT!")
print("🚨" + "="*80 + "🚨")

🚨================================================================================🚨
             SEBI FRAUD DETECTION SYSTEM - COMPREHENSIVE REPORT
🚨================================================================================🚨

🔍 CONTENT SCANS PERFORMED: 4

   Scan #1:
   ⚠️  Risk Score: 0/100
   🔍 Patterns: 
   📝 Content: 🚀 URGENT: Buy XYZ Corp before announcement tomorrow!
Insider information confirms 500% jump!
Trust me, I made ₹50 lakhs last week!
Delete after reading!

   Scan #2:
   ⚠️  Risk Score: 30/100
   🔍 Patterns: secret algorithm, risk-free, act now
   📝 Content: 🔥 GUARANTEED 800% RETURNS in 45 days! 🔥
Join our EXCLUSIVE trading group with SECRET ALGORITHMS used by billionaires!
✅ Risk-FREE investment ✅ No experience needed
📞 Contact certified advisor: Rajesh ...

   Scan #3:
   ⚠️  Risk Score: 0/100
   🔍 Patterns: 
   📝 Content: 📱 Download TradeGuru Pro - India's #1 trading platform!
⭐ 5-star ratings ⭐ Used by 10 lakh+ investors
🎁 Sign up bonus: ₹10,000 free trading cr

## 🌐 Launch Web Interface

The fraud detection system includes a **Google-like web interface** for easy access by regulators, investors, and compliance teams. The interface provides:

### 📱 **Key Features:**
- **🔍 Content Scanner**: Real-time fraud detection and analysis
- **✅ Advisor Verification**: SEBI credential validation
- **📱 Social Media Monitor**: Platform monitoring dashboard  
- **📊 Corporate Announcements**: Authenticity verification
- **⚠️ Risk Assessment**: Comprehensive threat analysis

### 🚀 **To Launch the Web Interface:**

1. **Install requirements**: `pip install -r web_requirements.txt`
2. **Set up environment**: Add your `GEMINI_API_KEY` to `.env` file
3. **Run the launcher**: `python run_web_app.py`
4. **Access interface**: Open http://localhost:8501 in your browser

The web interface provides an intuitive, Google-inspired design that makes fraud detection accessible to all stakeholders while maintaining enterprise-grade security and compliance standards.

In [ ]:
# Launch the SEBI Fraud Detection Web Interface
import subprocess
import sys
import os
from pathlib import Path

def launch_fraud_detection_web():
    """Launch the fraud detection web interface"""
    
    print("🛡️  SEBI FRAUD DETECTION SYSTEM")
    print("=" * 60)
    print("🌐 Google-like Web Interface Ready!")
    print("📊 Features: Content Scanner, Advisor Verification, Social Media Monitor")
    print("🔍 AI-Powered: Gemini 2.5 Pro fraud detection")
    print("=" * 60)
    
    # Check if web files exist
    web_app_path = Path("fraud_detection_web.py")
    launcher_path = Path("run_web_app.py")
    
    if web_app_path.exists() and launcher_path.exists():
        print("✅ Web interface files found!")
        print("🚀 To launch the web app, run:")
        print("   📝 python run_web_app.py")
        print("   🌐 Then open: http://localhost:8501")
        print("\n🔧 Prerequisites:")
        print("   📦 pip install streamlit plotly pandas")
        print("   🔑 Set GEMINI_API_KEY in environment")
        
        # Option to launch directly (commented out for safety)
        """
        launch_choice = input("\\n🚀 Launch web interface now? (y/n): ")
        if launch_choice.lower() == 'y':
            try:
                subprocess.run([sys.executable, "run_web_app.py"])
            except Exception as e:
                print(f"❌ Error: {e}")
        """
        
    else:
        print("❌ Web interface files not found!")
        print("💡 Make sure fraud_detection_web.py and run_web_app.py are in the current directory")
    
    print("\n🎯 Web Interface Features:")
    print("- 🔍 Real-time content fraud scanning")
    print("- ✅ SEBI advisor credential verification")
    print("- 📱 Social media monitoring dashboard")
    print("- 📊 Corporate announcement analysis")
    print("- ⚠️ Comprehensive risk assessment")
    print("- 🛡️  Google-inspired design for ease of use")

# Run the launch function
launch_fraud_detection_web()